<a href="https://colab.research.google.com/github/IraitzM/dlt_homeworks/blob/main/Workshop_Part1_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Homework

Write a python script that will load data from the SpaceX API into DuckDB using dlt.

Use:
- @dlt.source
- @dlt.resource
- @dlt.transformer

SpaceX API URL: https://api.spacexdata.com

Docs: https://github.com/r-spacex/SpaceX-API/blob/master/docs/README.md

Endpoints for loading:
- launches
- rockets
- crew

# Install dlt with duckdb extention

In [1]:
%%capture
!pip install dlt[duckdb]

# Play with SpaceX API

In [31]:
import requests
response = requests.get("https://api.spacexdata.com/v4/launches")
response.json()[0]

{'fairings': {'reused': False,
  'recovery_attempt': False,
  'recovered': False,
  'ships': []},
 'links': {'patch': {'small': 'https://images2.imgbox.com/94/f2/NN6Ph45r_o.png',
   'large': 'https://images2.imgbox.com/5b/02/QcxHUb5V_o.png'},
  'reddit': {'campaign': None,
   'launch': None,
   'media': None,
   'recovery': None},
  'flickr': {'small': [], 'original': []},
  'presskit': None,
  'webcast': 'https://www.youtube.com/watch?v=0a_00nJ_Y88',
  'youtube_id': '0a_00nJ_Y88',
  'article': 'https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html',
  'wikipedia': 'https://en.wikipedia.org/wiki/DemoSat'},
 'static_fire_date_utc': '2006-03-17T00:00:00.000Z',
 'static_fire_date_unix': 1142553600,
 'net': False,
 'window': 0,
 'rocket': '5e9d0d95eda69955f709d1eb',
 'success': False,
 'failures': [{'time': 33,
   'altitude': None,
   'reason': 'merlin engine failure'}],
 'details': 'Engine failure at 33 seconds and loss of vehicle',
 'crew': [],
 'ships': [],
 'caps

In [30]:
import requests
response = requests.get("https://api.spacexdata.com/v4/rockets")
response.json()[0]

{'height': {'meters': 22.25, 'feet': 73},
 'diameter': {'meters': 1.68, 'feet': 5.5},
 'mass': {'kg': 30146, 'lb': 66460},
 'first_stage': {'thrust_sea_level': {'kN': 420, 'lbf': 94000},
  'thrust_vacuum': {'kN': 480, 'lbf': 110000},
  'reusable': False,
  'engines': 1,
  'fuel_amount_tons': 44.3,
  'burn_time_sec': 169},
 'second_stage': {'thrust': {'kN': 31, 'lbf': 7000},
  'payloads': {'composite_fairing': {'height': {'meters': 3.5, 'feet': 11.5},
    'diameter': {'meters': 1.5, 'feet': 4.9}},
   'option_1': 'composite fairing'},
  'reusable': False,
  'engines': 1,
  'fuel_amount_tons': 3.38,
  'burn_time_sec': 378},
 'engines': {'isp': {'sea_level': 267, 'vacuum': 304},
  'thrust_sea_level': {'kN': 420, 'lbf': 94000},
  'thrust_vacuum': {'kN': 480, 'lbf': 110000},
  'number': 1,
  'type': 'merlin',
  'version': '1C',
  'layout': 'single',
  'engine_loss_max': 0,
  'propellant_1': 'liquid oxygen',
  'propellant_2': 'RP-1 kerosene',
  'thrust_to_weight': 96},
 'landing_legs': {'numb

# Helper
Run the cell and ignore it.

In [3]:
from dlt.common.pipeline import LoadInfo

def assert_load_info(info: LoadInfo, expected_load_packages: int = 1) -> None:
    """Asserts that expected number of packages was loaded and there are no failed jobs"""
    assert len(info.loads_ids) == expected_load_packages
    # all packages loaded
    assert all(package.state == "loaded" for package in info.load_packages) is True
    # no failed jobs in any of the packages
    info.raise_on_failed_jobs()

# Task 1


Create a pipeline for SpaceX API, for the next endpoints: launches, rockets, crew.

- Fill the empty lines in the functions below.
- `get_rockets` resource should have `table_name=rockets`.
- Create a [resource](https://dlthub.com/docs/general-usage/resource#declare-a-resource) for the `crew` endpoint from scratch.
- [Run the pipeline](https://dlthub.com/docs/walkthroughs/run-a-pipeline) without errors.

In [4]:
import time

import dlt
import requests


@dlt.resource(table_name="launches")
def get_launches():
    # put your code here
    response = requests.get("https://api.spacexdata.com/v4/launches")
    resp_json = response.json()
    for rjson in resp_json:
      yield rjson

@dlt.resource(table_name="rockets")
def get_rockets():
    # put your code here
    response = requests.get("https://api.spacexdata.com/v4/rockets")
    resp_json = response.json()
    for rjson in resp_json:
      yield rjson


@dlt.resource(table_name="crew")
def get_crew():
  # put your code here
  response = requests.get("https://api.spacexdata.com/v4/crew")
  resp_json = response.json()
  for rjson in resp_json:
    yield rjson


pipeline = dlt.pipeline(
    pipeline_name='spacex_with_source',
    destination='duckdb',
    dataset_name='spacex_data',
    dev_mode=True,
)

load_info = pipeline.run([get_launches(), get_rockets(), get_crew()])
print(load_info)
assert_load_info(load_info)


Pipeline spacex_with_source load step completed in 2.32 seconds
1 load package(s) were loaded to destination duckdb and into dataset spacex_data_20240823065616
The duckdb destination used duckdb:////content/spacex_with_source.duckdb location to store data
Load package 1724396176.1081264 is LOADED and contains no failed jobs


Run the code below and
## Answer the Question:
- What weight (kg) has the heighest (meters) rocket?

In [6]:
import duckdb
from google.colab import data_table
data_table.enable_dataframe_formatter()

# a database '<pipeline_name>.duckdb' was created in working directory so just connect to it
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
stats_table = conn.sql("SELECT mass__kg FROM rockets ORDER BY height__meters DESC").df()
display(stats_table)

,mass__kg
0,1335000
1,549054
2,1420788
3,30146


# Task 2
- Add pagination, read [SpaceX API doc](https://github.com/r-spacex/SpaceX-API/blob/master/docs/queries.md).
- Combine all resources in one [source](https://dlthub.com/docs/general-usage/source) and the pipeline with `@dlt.source`.
- Add incremental loading for resource `get_launches` using `merge` write disposition, `id` as a  primary key and `dlt.sources.incremental`.
- Run the pipeline [only with](https://dlthub.com/docs/general-usage/source#access-and-select-resources-to-load) `get_launches` resource.

Read more about [incremental loading](https://dlthub.com/docs/general-usage/incremental-loading).

## Try post method to query SpaceX API


In [19]:
import requests

response = requests.post(
    "https://api.spacexdata.com/v4/launches/query",
    json={
        "query": {
            "date_unix": {
                "$gt": 0
            },
        },
        "options": {
            "page": 1
        }
    }
)
response.json().keys()

dict_keys(['docs', 'totalDocs', 'limit', 'totalPages', 'page', 'pagingCounter', 'hasPrevPage', 'hasNextPage', 'prevPage', 'nextPage'])

In [20]:
response.json()

{'docs': [{'fairings': {'reused': False,
    'recovery_attempt': False,
    'recovered': False,
    'ships': []},
   'links': {'patch': {'small': 'https://images2.imgbox.com/94/f2/NN6Ph45r_o.png',
     'large': 'https://images2.imgbox.com/5b/02/QcxHUb5V_o.png'},
    'reddit': {'campaign': None,
     'launch': None,
     'media': None,
     'recovery': None},
    'flickr': {'small': [], 'original': []},
    'presskit': None,
    'webcast': 'https://www.youtube.com/watch?v=0a_00nJ_Y88',
    'youtube_id': '0a_00nJ_Y88',
    'article': 'https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html',
    'wikipedia': 'https://en.wikipedia.org/wiki/DemoSat'},
   'static_fire_date_utc': '2006-03-17T00:00:00.000Z',
   'static_fire_date_unix': 1142553600,
   'net': False,
   'window': 0,
   'rocket': '5e9d0d95eda69955f709d1eb',
   'success': False,
   'failures': [{'time': 33,
     'altitude': None,
     'reason': 'merlin engine failure'}],
   'details': 'Engine failure at 33 sec

Use code above to make your launches resource incremental. Use date_unix 0 as an initial value.

In [47]:
import dlt
import requests

def pagination(date_unix, page = 0):
    while True:
        response = requests.post(
            "https://api.spacexdata.com/v4/launches/query",
            json={
                "query": {
                    "date_unix": {
                        "$gt": date_unix,
                    },
                },
                "options": {
                    "page": page
                }
            }
        )
        response.raise_for_status()
        yield from response.json()['docs']

        # Get next page
        page = response.json()["nextPage"]
        if not page:
            break

@dlt.resource(
    table_name="launches",
    primary_key="id",
    write_disposition="merge",

)
def get_launches(date_unix = dlt.sources.incremental("date_unix", initial_value = 0)):
    # put your code here
    yield pagination(date_unix.start_value)


@dlt.resource(table_name="rockets")
def get_rockets():
    # put your code here
    response = requests.get("https://api.spacexdata.com/v4/rockets")
    resp_json = response.json()
    for rjson in resp_json:
      yield rjson


@dlt.resource(table_name="crew")
def get_crew():
  # put your code here
  response = requests.get("https://api.spacexdata.com/v4/crew")
  resp_json = response.json()
  for rjson in resp_json:
    yield rjson


@dlt.source
def spacex_source():
    return [get_launches, get_rockets, get_crew]


pipeline = dlt.pipeline(
    pipeline_name='spacex_with_source_inc',
    destination='duckdb',
    dataset_name='spacex_data_inc',
    dev_mode=True,
)

In [48]:
data = spacex_source().resources['get_launches'](1641000000)

load_info = pipeline.run(data)
print(load_info)
assert_load_info(load_info)

# Run the pipeline one more time, it should load no data
load_info = pipeline.run(data)
print(load_info)
assert_load_info(load_info, expected_load_packages=0)

Pipeline spacex_with_source_inc load step completed in 2.25 seconds
1 load package(s) were loaded to destination duckdb and into dataset spacex_data_inc_20240823073957
The duckdb destination used duckdb:////content/spacex_with_source_inc.duckdb location to store data
Load package 1724398799.3682308 is LOADED and contains no failed jobs
Pipeline spacex_with_source_inc load step completed in ---
0 load package(s) were loaded to destination duckdb and into dataset None
The duckdb destination used duckdb:////content/spacex_with_source_inc.duckdb location to store data


## Answer the Question:
What rocket was launched between 2022-11-01 and 2022-11-02?


In [49]:
import duckdb
from google.colab import data_table
data_table.enable_dataframe_formatter()

# a database '<pipeline_name>.duckdb' was created in working directory so just connect to it
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
stats_table = conn.sql("SELECT * FROM launches WHERE date_utc BETWEEN '2022-11-01 00:00:00+00:00' AND '2022-11-02 00:00:00+00:00'").df()
display(stats_table)

,links__patch__small,links__patch__large,links__reddit__launch,links__reddit__recovery,links__webcast,links__youtube_id,net,rocket,success,launchpad,...,id,_dlt_load_id,_dlt_id,links__article,links__reddit__campaign,static_fire_date_utc,static_fire_date_unix,details,fairings__recovery_attempt,links__wikipedia
0,None,None,None,None,https://youtu.be/pY628jRd6gM,pY628jRd6gM,False,5e9d0d95eda69974db09d1ed,NaN,5e9e4502f509094188566f88,...,6243aec2af52800c6e91925d,1724398799.3682308,ymJVGdBc2FKvZw,None,None,NaT,NaN,None,NaN,None
1,None,None,None,None,None,None,False,5e9d0d95eda69973a809d1ec,NaN,5e9e4501f509094ba4566f84,...,6243ba08af52800c6e919270,1724398799.3682308,+rptI+T3QjaiRA,None,None,NaT,NaN,None,NaN,None
2,https://images2.imgbox.com/a9/9a/NXVkTZCE_o.png,https://images2.imgbox.com/e3/cc/hN96PmST_o.png,None,https://www.reddit.com/r/spacex/comments/k2ts1...,https://youtu.be/wcq7xiTOPRg,wcq7xiTOPRg,False,5e9d0d95eda69973a809d1ec,NaN,5e9e4501f509094ba4566f84,...,633f71820531f07b4fdf59bd,1724398799.3682308,qbfGj3kpXbRY4g,None,https://www.reddit.com/r/spacex/comments/jhu37...,NaT,NaN,None,NaN,None
3,None,None,None,None,None,None,False,5e9d0d95eda69973a809d1ec,NaN,5e9e4501f509094ba4566f84,...,633f72000531f07b4fdf59c2,1724398799.3682308,kFnQG0nfKaPYJA,None,None,NaT,NaN,None,NaN,None


# Task 3: Get payloads of launches

Use `@dlt.transformer` to get additional info for your data.

Read more about dlt [transformers](https://dlthub.com/docs/general-usage/resource#process-resources-with-dlttransformer).

In [71]:
import dlt
import requests

@dlt.resource(table_name="launches")
def get_launches():
    # put your code here
    response = requests.get("https://api.spacexdata.com/v4/launches")
    resp_json = response.json()
    for rjson in resp_json:
      yield rjson

@dlt.transformer(data_from=get_launches, table_name="details")
def get_details(items):
    yield items["details"]

pipeline = dlt.pipeline(
    pipeline_name='spacex_with_source_tr',
    destination='duckdb',
    dataset_name='spacex_data_tr',
    dev_mode=True,
)

data = get_launches | get_details

load_info = pipeline.run(data())
print(load_info)
assert_load_info(load_info)


Pipeline spacex_with_source_tr load step completed in 1.11 seconds
1 load package(s) were loaded to destination duckdb and into dataset spacex_data_tr_20240823075119
The duckdb destination used duckdb:////content/spacex_with_source_tr.duckdb location to store data
Load package 1724399479.7459087 is LOADED and contains no failed jobs


In [72]:
import duckdb
from google.colab import data_table
data_table.enable_dataframe_formatter()

# a database '<pipeline_name>.duckdb' was created in working directory so just connect to it
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
# list all tables
display(conn.sql("DESCRIBE"))

┌──────────────────────┬──────────────────────┬─────────────────────┬───┬──────────────────────┬───────────┐
│       database       │        schema        │        name         │ … │     column_types     │ temporary │
│       varchar        │       varchar        │       varchar       │   │      varchar[]       │  boolean  │
├──────────────────────┼──────────────────────┼─────────────────────┼───┼──────────────────────┼───────────┤
│ spacex_with_source…  │ spacex_data_tr_202…  │ _dlt_loads          │ … │ [VARCHAR, VARCHAR,…  │ false     │
│ spacex_with_source…  │ spacex_data_tr_202…  │ _dlt_pipeline_state │ … │ [BIGINT, BIGINT, V…  │ false     │
│ spacex_with_source…  │ spacex_data_tr_202…  │ _dlt_version        │ … │ [BIGINT, BIGINT, T…  │ false     │
│ spacex_with_source…  │ spacex_data_tr_202…  │ get_payloads        │ … │ [VARCHAR, VARCHAR,…  │ false     │
│ spacex_with_source…  │ spacex_data_tr_202…  │ _dlt_loads          │ … │ [VARCHAR, VARCHAR,…  │ false     │
│ spacex_with_sourc

Run the code below and

## Answer the Question:

What regime has Satellite "FalconSAT-2" with launch id: 5eb87cd9ffd86e000604b32a?  

In [73]:
stats_table = conn.sql("SELECT * FROM details").df()
display(stats_table)

,value,_dlt_load_id,_dlt_id
0,Engine failure at 33 seconds and loss of vehicle,1724399479.7459087,vuXFNHz0A38vYw
1,Successful first stage burn and transition to ...,1724399479.7459087,7mDQ2hl0ZmvqxA
2,Residual stage 1 thrust led to collision betwe...,1724399479.7459087,Er4+rdZKWa6zgA
3,Ratsat was carried to orbit on the first succe...,1724399479.7459087,ExysQJvPkJO9pA
4,"Launch was scrubbed on first attempt, second l...",1724399479.7459087,QXco3mvSbApx1Q
...,...,...,...
129,The mission consists in launching 52 Starlink ...,1724399479.7459087,+u6ohuSWmM6u9A
130,"The Türksat 5B communication satellite, which ...",1724399479.7459087,GXhbjs0Wnhljdw
131,SpaceX's 24th ISS resupply mission on behalf o...,1724399479.7459087,pTXyAxbsJCpBPA
132,Falcon 9 launches to sun-synchronous polar orb...,1724399479.7459087,Rr1PAcDQASyWew
